In [125]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [126]:
df = pd.read_csv("./bases/new_complete/complete_raw.csv", low_memory=False)

In [127]:
df = df[df["status"] == "Released"]

In [128]:
df[['Title', 'Ratings']].head()

,Title,Ratings
0,Femme,"[{""Source"": ""Internet Movie Database"", ""Value""..."
1,Goebbels and the Führer,"[{""Source"": ""Internet Movie Database"", ""Value""..."
2,Skincare,"[{""Source"": ""Internet Movie Database"", ""Value""..."
3,Nightwatch: Demons Are Forever,"[{""Source"": ""Internet Movie Database"", ""Value""..."
4,Moxie,"[{""Source"": ""Internet Movie Database"", ""Value""..."


In [129]:
df.columns

Index(['Unnamed: 0', 'id', 'value_x', 'Title', 'Year', 'Rated', 'Released',
       'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language',
       'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response', 'Error', 'Season', 'Episode', 'seriesID', 'id_x',
       'imdb_id', 'value_y', 'adult', 'backdrop_path', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id_y', 'origin_country',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'belongs_to_collection.id', 'belongs_to_collection.name',
       'belongs_to_collection.poster_path',
       'belongs_to_collection.backdrop_path'],
      dtype='object')

In [130]:
#columns_to_ignore = [
#    "Poster", "Awards", "Season","video","DVD",
#    "homepage","poster_path","tagline","Website","Plot","Episode","Season","overview",   
#]

columns_to_use = {
    "id", "Title", "Year", "Ratings", "runtime", "Genre", "genres",
    "Ratings", "budget", 'revenue', 'popularity', "production_companies", "BoxOffice",
    "original_language",
}

columns_to_drop = set(df.columns) - columns_to_use

df = df.drop(columns_to_drop, axis=1)

In [131]:
df

,id,Title,Year,Genre,Ratings,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime
0,tt20114686,Femme,2023,"Drama, Thriller","[{""Source"": ""Internet Movie Database"", ""Value""...","$187,053",NaN,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",en,2.8926,"[{""id"": 85136, ""logo_path"": ""/gkOBQC95ZhfwC1KK...",0,99
1,tt30467885,Goebbels and the Führer,2024,"Biography, Drama, History","[{""Source"": ""Internet Movie Database"", ""Value""...",NaN,NaN,0,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",de,2.1574,"[{""id"": 35389, ""logo_path"": ""/nX4hlwsvFV7IhhpK...",547033,123
2,tt28223926,Skincare,2024,"Comedy, Horror, Mystery","[{""Source"": ""Internet Movie Database"", ""Value""...","$456,590",NaN,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 18, ""n...",en,5.4150,"[{""id"": 232460, ""logo_path"": null, ""name"": ""Ja...",451381,96
3,tt6318608,Nightwatch: Demons Are Forever,2023,"Horror, Mystery, Thriller","[{""Source"": ""Internet Movie Database"", ""Value""...",NaN,NaN,4300000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",da,33.0393,"[{""id"": 235, ""logo_path"": ""/obOynoztBEhDNfHsDX...",0,117
4,tt6432466,Moxie,2021,"Comedy, Drama","[{""Source"": ""Internet Movie Database"", ""Value""...",NaN,NaN,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",en,3.1624,"[{""id"": 24022, ""logo_path"": ""/aIjT1pYb83Hluuq3...",0,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9682,tt9887520,Season of Love,2019,"Comedy, Romance","[{""Source"": ""Internet Movie Database"", ""Value""...",NaN,NaN,0,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ...",en,0.6198,"[{""id"": 119154, ""logo_path"": ""/97OWtUjBFLxtztM...",0,105
9683,tt9894470,VFW,2019,"Action, Crime, Horror","[{""Source"": ""Internet Movie Database"", ""Value""...",NaN,NaN,1000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",en,1.1559,"[{""id"": 53472, ""logo_path"": ""/o8znB0tc9qvtdQLC...",23101,92
9684,tt9896916,Pilgrim's Progress,2019,"Animation, Adventure, Family","[{""Source"": ""Internet Movie Database"", ""Value""...","$1,294,596",NaN,0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 12, ""...",en,1.3392,"[{""id"": 117130, ""logo_path"": null, ""name"": ""Ca...",0,108
9685,tt9900782,Kaithi,2019,"Action, Crime, Drama","[{""Source"": ""Internet Movie Database"", ""Value""...",NaN,NaN,0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",ta,1.6570,"[{""id"": 16429, ""logo_path"": ""/8VoXlyno166HSIwM...",0,147


# Ratings

In [132]:
import ast

# Function to convert list of dicts into a dict
def extract_ratings(ratings_list):
    return {item['Source']: item['Value'] for item in ratings_list}

#df['Ratings'].apply(json.loads).apply(extract_ratings).apply(pd.Series)

df = pd.concat([
    df.drop(columns=["Ratings"]),
    df['Ratings'].apply(json.loads).apply(extract_ratings).apply(pd.Series)
], axis=1)

In [133]:
df.head()

,id,Title,Year,Genre,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime,Internet Movie Database,Rotten Tomatoes,Metacritic
0,tt20114686,Femme,2023,"Drama, Thriller","$187,053",NaN,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",en,2.8926,"[{""id"": 85136, ""logo_path"": ""/gkOBQC95ZhfwC1KK...",0,99,7.3/10,93%,69/100
1,tt30467885,Goebbels and the Führer,2024,"Biography, Drama, History",NaN,NaN,0,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",de,2.1574,"[{""id"": 35389, ""logo_path"": ""/nX4hlwsvFV7IhhpK...",547033,123,6.7/10,NaN,NaN
2,tt28223926,Skincare,2024,"Comedy, Horror, Mystery","$456,590",NaN,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 18, ""n...",en,5.4150,"[{""id"": 232460, ""logo_path"": null, ""name"": ""Ja...",451381,96,5.7/10,69%,NaN
3,tt6318608,Nightwatch: Demons Are Forever,2023,"Horror, Mystery, Thriller",NaN,NaN,4300000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",da,33.0393,"[{""id"": 235, ""logo_path"": ""/obOynoztBEhDNfHsDX...",0,117,5.8/10,69%,50/100
4,tt6432466,Moxie,2021,"Comedy, Drama",NaN,NaN,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",en,3.1624,"[{""id"": 24022, ""logo_path"": ""/aIjT1pYb83Hluuq3...",0,111,6.7/10,70%,54/100


In [134]:
def extract_companies(companies_list):
    names = [item['name'] for item in companies_list]
    return ','.join(names)

df["production_companies"] = df["production_companies"].apply(json.loads).apply(extract_companies)

In [162]:
df['production_companies'] = df['production_companies'].apply(lambda x : x.split(','))

In [135]:
df.head()

,id,Title,Year,Genre,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime,Internet Movie Database,Rotten Tomatoes,Metacritic
0,tt20114686,Femme,2023,"Drama, Thriller","$187,053",NaN,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",en,2.8926,Agile Films,0,99,7.3/10,93%,69/100
1,tt30467885,Goebbels and the Führer,2024,"Biography, Drama, History",NaN,NaN,0,"[{""id"": 36, ""name"": ""History""}, {""id"": 18, ""na...",de,2.1574,"Zeitsprung Pictures,SWR,act headquarter media",547033,123,6.7/10,NaN,NaN
2,tt28223926,Skincare,2024,"Comedy, Horror, Mystery","$456,590",NaN,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 18, ""n...",en,5.4150,"Jalapeno Goat,Iervolino & Lady Bacardi Enterta...",451381,96,5.7/10,69%,NaN
3,tt6318608,Nightwatch: Demons Are Forever,2023,"Horror, Mystery, Thriller",NaN,NaN,4300000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",da,33.0393,Nordisk Film Denmark,0,117,5.8/10,69%,50/100
4,tt6432466,Moxie,2021,"Comedy, Drama",NaN,NaN,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",en,3.1624,Paper Kite Productions,0,111,6.7/10,70%,54/100


# Handle Genres

In [140]:
# Function to convert list of dicts into a dict
def extract_genres(genres_list):
    return [ item['name'] for item in genres_list ]

df['genres'] = df['genres'].apply(json.loads).apply(extract_genres)

In [141]:

df = df[df['Genre'].apply(lambda x: isinstance(x, str))]

In [142]:
df['Genre'] = df['Genre'].apply(lambda line : [ s.strip() for s in line.split(',') ])

In [146]:
df.head()

,id,Title,Year,Genre,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime,Internet Movie Database,Rotten Tomatoes,Metacritic
0,tt20114686,Femme,2023,"[Drama, Thriller]","$187,053",NaN,0,"[Drama, Thriller, Romance]",en,2.8926,Agile Films,0,99,7.3/10,93%,69/100
1,tt30467885,Goebbels and the Führer,2024,"[Biography, Drama, History]",NaN,NaN,0,"[History, Drama]",de,2.1574,"Zeitsprung Pictures,SWR,act headquarter media",547033,123,6.7/10,NaN,NaN
2,tt28223926,Skincare,2024,"[Comedy, Horror, Mystery]","$456,590",NaN,0,"[Thriller, Drama]",en,5.4150,"Jalapeno Goat,Iervolino & Lady Bacardi Enterta...",451381,96,5.7/10,69%,NaN
3,tt6318608,Nightwatch: Demons Are Forever,2023,"[Horror, Mystery, Thriller]",NaN,NaN,4300000,"[Horror, Thriller, Mystery]",da,33.0393,Nordisk Film Denmark,0,117,5.8/10,69%,50/100
4,tt6432466,Moxie,2021,"[Comedy, Drama]",NaN,NaN,0,"[Comedy, Drama, Music]",en,3.1624,Paper Kite Productions,0,111,6.7/10,70%,54/100


In [148]:
df['generoos'] = df.apply(lambda row: list(set(row['genres']) | set(row['Genre'])), axis=1)

In [150]:
df[['generoos', 'Genre', 'genres']]

,generoos,Genre,genres
0,"[Thriller, Romance, Drama]","[Drama, Thriller]","[Drama, Thriller, Romance]"
1,"[History, Drama, Biography]","[Biography, Drama, History]","[History, Drama]"
2,"[Thriller, Horror, Comedy, Mystery, Drama]","[Comedy, Horror, Mystery]","[Thriller, Drama]"
3,"[Thriller, Horror, Mystery]","[Horror, Mystery, Thriller]","[Horror, Thriller, Mystery]"
4,"[Comedy, Drama, Music]","[Comedy, Drama]","[Comedy, Drama, Music]"
...,...,...,...
9682,"[Romance, Comedy]","[Comedy, Romance]","[Romance, Comedy]"
9683,"[Action, Thriller, Horror, Crime]","[Action, Crime, Horror]","[Action, Crime, Horror, Thriller]"
9684,"[Adventure, Family, Animation]","[Animation, Adventure, Family]","[Animation, Adventure]"
9685,"[Action, Thriller, Drama, Crime]","[Action, Crime, Drama]","[Action, Thriller, Crime]"


In [152]:
df['genres'] = df['generoos']

df = df.drop(columns=['Genre', 'generoos'])

,id,Title,Year,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime,Internet Movie Database,Rotten Tomatoes,Metacritic
0,tt20114686,Femme,2023,"$187,053",NaN,0,"[Thriller, Romance, Drama]",en,2.8926,Agile Films,0,99,7.3/10,93%,69/100
1,tt30467885,Goebbels and the Führer,2024,NaN,NaN,0,"[History, Drama, Biography]",de,2.1574,"Zeitsprung Pictures,SWR,act headquarter media",547033,123,6.7/10,NaN,NaN
2,tt28223926,Skincare,2024,"$456,590",NaN,0,"[Thriller, Horror, Comedy, Mystery, Drama]",en,5.4150,"Jalapeno Goat,Iervolino & Lady Bacardi Enterta...",451381,96,5.7/10,69%,NaN
3,tt6318608,Nightwatch: Demons Are Forever,2023,NaN,NaN,4300000,"[Thriller, Horror, Mystery]",da,33.0393,Nordisk Film Denmark,0,117,5.8/10,69%,50/100
4,tt6432466,Moxie,2021,NaN,NaN,0,"[Comedy, Drama, Music]",en,3.1624,Paper Kite Productions,0,111,6.7/10,70%,54/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9682,tt9887520,Season of Love,2019,NaN,NaN,0,"[Romance, Comedy]",en,0.6198,"Tello Films,DASH Productions",0,105,6.6/10,NaN,NaN
9683,tt9894470,VFW,2019,NaN,NaN,1000000,"[Action, Thriller, Horror, Crime]",en,1.1559,"Fangoria Films,Channel 83 Films,Good Wizard,Vo...",23101,92,6.1/10,85%,72/100
9684,tt9896916,Pilgrim's Progress,2019,"$1,294,596",NaN,0,"[Adventure, Family, Animation]",en,1.3392,"Cat In The Mill Studio,Herald Entertainment",0,108,6.3/10,60%,NaN
9685,tt9900782,Kaithi,2019,NaN,NaN,0,"[Action, Thriller, Drama, Crime]",ta,1.6570,"Dream Warrior Pictures,Vivekandanda Pictures",0,147,8.4/10,100%,NaN


In [156]:
df

,id,Title,Year,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime,Internet Movie Database,Rotten Tomatoes,Metacritic
0,tt20114686,Femme,2023,"$187,053",NaN,0,"[Thriller, Romance, Drama]",en,2.8926,Agile Films,0,99,7.3/10,93%,69/100
1,tt30467885,Goebbels and the Führer,2024,NaN,NaN,0,"[History, Drama, Biography]",de,2.1574,"Zeitsprung Pictures,SWR,act headquarter media",547033,123,6.7/10,NaN,NaN
2,tt28223926,Skincare,2024,"$456,590",NaN,0,"[Thriller, Horror, Comedy, Mystery, Drama]",en,5.4150,"Jalapeno Goat,Iervolino & Lady Bacardi Enterta...",451381,96,5.7/10,69%,NaN
3,tt6318608,Nightwatch: Demons Are Forever,2023,NaN,NaN,4300000,"[Thriller, Horror, Mystery]",da,33.0393,Nordisk Film Denmark,0,117,5.8/10,69%,50/100
4,tt6432466,Moxie,2021,NaN,NaN,0,"[Comedy, Drama, Music]",en,3.1624,Paper Kite Productions,0,111,6.7/10,70%,54/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9682,tt9887520,Season of Love,2019,NaN,NaN,0,"[Romance, Comedy]",en,0.6198,"Tello Films,DASH Productions",0,105,6.6/10,NaN,NaN
9683,tt9894470,VFW,2019,NaN,NaN,1000000,"[Action, Thriller, Horror, Crime]",en,1.1559,"Fangoria Films,Channel 83 Films,Good Wizard,Vo...",23101,92,6.1/10,85%,72/100
9684,tt9896916,Pilgrim's Progress,2019,"$1,294,596",NaN,0,"[Adventure, Family, Animation]",en,1.3392,"Cat In The Mill Studio,Herald Entertainment",0,108,6.3/10,60%,NaN
9685,tt9900782,Kaithi,2019,NaN,NaN,0,"[Action, Thriller, Drama, Crime]",ta,1.6570,"Dream Warrior Pictures,Vivekandanda Pictures",0,147,8.4/10,100%,NaN


# Ratings

In [194]:
df['imdb'] = df['Internet Movie Database']
df = df.drop(columns=['Internet Movie Database'])

In [196]:
df['rotten'] = df['Rotten Tomatoes']
df = df.drop(columns=['Rotten Tomatoes'])

In [205]:
df['imdb'] = df['imdb'].apply(lambda s : s.replace('/10', '').replace('.', '') if pd.notna(s) else s).apply(lambda s : int(s) if pd.notna(s) else s)

In [208]:
df['rotten'] = df['rotten'].apply(lambda s : s.replace('%', '') if pd.notna(s) else s).apply(lambda s : int(s) if pd.notna(s) else s)

In [216]:
df['Metacritic'] = df['Metacritic'].apply(lambda s : s.replace('/100', '') if pd.notna(s) else s).apply(lambda s : int(s) if pd.notna(s) else s)

In [217]:
df

,id,Title,Year,BoxOffice,Production,budget,genres,original_language,popularity,production_companies,revenue,runtime,Metacritic,imdb,rotten
0,tt20114686,Femme,2023,"$187,053",NaN,0,"[Thriller, Romance, Drama]",en,2.8926,[Agile Films],0,99,69.0,73.0,93.0
1,tt30467885,Goebbels and the Führer,2024,NaN,NaN,0,"[History, Drama, Biography]",de,2.1574,"[Zeitsprung Pictures, SWR, act headquarter media]",547033,123,NaN,67.0,NaN
2,tt28223926,Skincare,2024,"$456,590",NaN,0,"[Thriller, Horror, Comedy, Mystery, Drama]",en,5.4150,"[Jalapeno Goat, Iervolino & Lady Bacardi Enter...",451381,96,NaN,57.0,69.0
3,tt6318608,Nightwatch: Demons Are Forever,2023,NaN,NaN,4300000,"[Thriller, Horror, Mystery]",da,33.0393,[Nordisk Film Denmark],0,117,50.0,58.0,69.0
4,tt6432466,Moxie,2021,NaN,NaN,0,"[Comedy, Drama, Music]",en,3.1624,[Paper Kite Productions],0,111,54.0,67.0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9682,tt9887520,Season of Love,2019,NaN,NaN,0,"[Romance, Comedy]",en,0.6198,"[Tello Films, DASH Productions]",0,105,NaN,66.0,NaN
9683,tt9894470,VFW,2019,NaN,NaN,1000000,"[Action, Thriller, Horror, Crime]",en,1.1559,"[Fangoria Films, Channel 83 Films, Good Wizard...",23101,92,72.0,61.0,85.0
9684,tt9896916,Pilgrim's Progress,2019,"$1,294,596",NaN,0,"[Adventure, Family, Animation]",en,1.3392,"[Cat In The Mill Studio, Herald Entertainment]",0,108,NaN,63.0,60.0
9685,tt9900782,Kaithi,2019,NaN,NaN,0,"[Action, Thriller, Drama, Crime]",ta,1.6570,"[Dream Warrior Pictures, Vivekandanda Pictures]",0,147,NaN,84.0,100.0


# SAve in CSV

In [218]:
# To fix how it store objects and lists
def is_complex(val):
    return isinstance(val, (list, dict))

def serialize_complex_columns(df):
    df_copy = df.copy()
    for col in df.columns:
        if df[col].apply(lambda x: is_complex(x)).any():
            df_copy[col] = df[col].apply(json.dumps)
    return df_copy

# Serialize and save
df_serialized = serialize_complex_columns(df)

In [219]:
df_serialized.to_csv('bases/definitive_dataset.csv')